# UAV Detection Using YOLO: Synthetic Data Training, Testing, and Result Visualization

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Download dataset from Roboflow Universe

In [2]:
from google.colab import userdata
secret = userdata.get('roboflowKey')

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=secret)
project = rf.workspace("ai-jbsna").project("drone3-c8zgs")
version = project.version(18)
dataset = version.download("yolov11")


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=secret)
project = rf.workspace("ai-jbsna").project("dronereal-3")
version = project.version(1)
dataset = version.download("yolov8")

**NOTE:**
Make sure the last 4 lines of the data.yaml file have the following format:

```
test: ../test/images
train: ../train/images
val: ../valid/images
```

If using a dataset from Roboflow, run the command below. 👇🏻

In [6]:
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml

## Download YOLO model

In [ ]:
pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11m.pt")

## Training YOLO model

In [ ]:

!yolo task=detect mode=train \
    model=/content/yolo11m.pt \
    data=/content/Drone3-18/data.yaml\
    epochs=200 \
    batch=16 \
    patience=50 \
    optimizer='AdamW' \
    lr0=0.01 \
    lrf=0.0001 \
    momentum=0.937 \
    weight_decay=0.0005 \
    warmup_epochs=3 \
    hsv_h=0.015 \
    hsv_s=0.8 \
    hsv_v=0.5 \
    degrees=15 \
    translate=0.2 \
    scale=0.6 \
    fliplr=0.5 \
    mosaic=1.0 \
    mixup=0.3 \
    copy_paste=0.3 \
    close_mosaic=10 \
    box=7.5 \
    cls=0.5 \
    dfl=1.5 \
    overlap_mask=True \
    mask_ratio=4 \
    seed=42

## Download YOLO folder with results

In [ ]:
import shutil
from google.colab import files

# Define the source folder and the zip file path
folder_path = '/content/runs/detect/train'
zip_path = '/content/yolo2002.zip'

# Compress the folder into a zip file
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

# Download the zip file to your laptop
files.download(zip_path)

## Visualize model results on test images

In [ ]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import yaml
import torch
from ultralytics import YOLO

def load_class_names(yaml_path):
    """Load class names from YAML file"""
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
        return data['names']

def plot_predictions_with_ground_truth(model, dataset, num_images=6):
    """
    Plot predictions and ground truth boxes for multiple images
    Args:
        model: YOLO model
        dataset: Dictionary containing paths
        num_images: Number of images to visualize
    """
    # Load class names
    class_names = load_class_names(dataset['data_yaml_path'])
    # Get list of image files
    image_files = list(Path(dataset['images_directory_path']).glob('*.jpg'))
    np.random.shuffle(image_files)
    # Create subplot grid
    fig, axes = plt.subplots(3, 2, figsize=(15, 10))
    axes = axes.ravel()
    for idx, img_path in enumerate(image_files[:num_images]):
        # Read image
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Get ground truth labels
        label_path = Path(dataset['annotations_directory_path']) / f"{img_path.stem}.txt"
        gt_boxes = []
        gt_classes = []
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x, y, w, h = map(float, line.strip().split())
                    gt_boxes.append([x, y, w, h])
                    gt_classes.append(int(class_id))
        # Get predictions
        results = model.predict(img, conf=0.25)[0]
        pred_boxes = results.boxes.xyxyn.cpu().numpy()  # normalized coordinates
        pred_classes = results.boxes.cls.cpu().numpy()
        pred_conf = results.boxes.conf.cpu().numpy()
        # Plot image
        axes[idx].imshow(img)
        # Initialize lists for legend handles and labels
        gt_handles = []
        pred_handles = []
        # Plot ground truth boxes
        for box, cls_id in zip(gt_boxes, gt_classes):
            x, y, w, h = box
            x1, y1 = x - w/2, y - h/2
            x2, y2 = x + w/2, y + h/2
            rect = plt.Rectangle((x1 * img.shape[1], y1 * img.shape[0]),
                               w * img.shape[1], h * img.shape[0],
                               fill=False, color='green', linewidth=2,
                               label=f'GT: {class_names[cls_id]}')
            axes[idx].add_patch(rect)
            gt_handles.append(rect)
        # Plot predicted boxes
        for box, cls_id, conf in zip(pred_boxes, pred_classes, pred_conf):
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1 * img.shape[1], y1 * img.shape[0]),
                               (x2-x1) * img.shape[1], (y2-y1) * img.shape[0],
                               fill=False, color='red', linewidth=2,
                               label=f'Pred: {class_names[int(cls_id)]} ({conf:.2f})')
            axes[idx].add_patch(rect)
            pred_handles.append(rect)
        axes[idx].set_title(f'Image {idx+1}')
        axes[idx].axis('off')
        # Add legend inside the image
        handles = gt_handles + pred_handles
        labels = [h.get_label() for h in handles]
        unique_labels = dict(zip(labels, handles))
        # Position the legend in the top-right corner with a white background and smaller font
        axes[idx].legend(unique_labels.values(), unique_labels.keys(),
                         loc='upper right',
                         bbox_to_anchor=(0.98, 0.98),
                         frameon=True,
                         facecolor='white',
                         edgecolor='lightgray',
                         prop={'size': 6})  # Reduced font size
    plt.tight_layout()
    plt.show()

# Example usage
dataset = {
    'images_directory_path': "/content/datasets/Drone3-18/test/images",
    'annotations_directory_path': "/content/datasets/Drone3-18/test/labels",
    'data_yaml_path': "/content/datasets/Drone3-18/data.yaml"
}

# Load the model
model = YOLO('content/YOLO200.pt')

# Plot predictions with ground truth
plot_predictions_with_ground_truth(model, dataset)


## Visualize model results on real images

In [ ]:
# Example usage
dataset = {
    'images_directory_path': "/content/datasets/DroneReal-2-4/test/images",
    'annotations_directory_path': "/content/datasets/DroneReal-2-4/test/labels",
    'data_yaml_path': "/content/datasets/DroneReal-2-4/data.yaml"
}

# Load the model
model = YOLO('/content/YOLO200.pt')

# Plot predictions with ground truth
plot_predictions_with_ground_truth(model, dataset)

## Testing YOLO model on test images

In [ ]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

def calculate_iou(box1, box2):
    """
    Calculate IoU between two boxes

    Args:
        box1: Array of shape (N, 4) containing N boxes in xyxy format
        box2: Array of shape (M, 4) containing M boxes in xyxy format

    Returns:
        IoU matrix of shape (N, M)
    """
    # Get coordinates
    b1_x1, b1_y1, b1_x2, b1_y2 = np.split(box1, 4, axis=1)
    b2_x1, b2_y1, b2_x2, b2_y2 = np.split(box2, 4, axis=1)

    # Calculate intersection coordinates
    x1 = np.maximum(b1_x1, np.transpose(b2_x1))
    y1 = np.maximum(b1_y1, np.transpose(b2_y1))
    x2 = np.minimum(b1_x2, np.transpose(b2_x2))
    y2 = np.minimum(b1_y2, np.transpose(b2_y2))

    # Calculate intersection area
    intersection = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)

    # Calculate union area
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    union = b1_area + np.transpose(b2_area) - intersection

    # Calculate IoU
    iou = intersection / (union + 1e-7)

    return iou

def calculate_ap50(confidences, labels, valid_labels):
    """
    Calculate Average Precision at IoU 0.50 for a single class

    Args:
        confidences (np.array): Confidence scores of predictions
        labels (np.array): Binary labels (1 for true positive, 0 for false positive)
        valid_labels (int): Number of ground truth instances

    Returns:
        float: Average Precision (AP50)
    """
    # Sort predictions by confidence in descending order
    sorted_indices = np.argsort(confidences)[::-1]
    sorted_labels = labels[sorted_indices]

    # Compute precision and recall
    tp = np.cumsum(sorted_labels)
    fp = np.cumsum(1 - sorted_labels)

    precision = tp / (tp + fp)
    recall = tp / max(valid_labels, 1)

    # Compute AP using 11-point interpolation
    ap = 0.0
    for t in np.linspace(0, 1, 11):
        # Find max precision for recalls >= t
        precisions_at_t = precision[recall >= t]
        max_precision = precisions_at_t.max() if len(precisions_at_t) > 0 else 0
        ap += max_precision / 11

    return ap

def evaluate_yolo_model(model_path, dataset, conf_threshold=0.25, iou_threshold=0.5):
    """
    Evaluate YOLO model performance on a dataset using multiple metrics

    Args:
        model_path (str): Path to the YOLO model weights
        dataset (sv.DetectionDataset): Supervision dataset object
        conf_threshold (float): Confidence threshold for predictions
        iou_threshold (float): IoU threshold for match determination
    """
    # Load model
    model = YOLO(model_path)

    # Initialize metrics storage
    metrics = {
        'precisions': [],
        'recalls': [],
        'f1_scores': [],
        'confidences': [],
        'true_positives': 0,
        'false_positives': 0,
        'false_negatives': 0,
        'predictions_by_class': defaultdict(list),
        'prediction_confidences_by_class': defaultdict(list),
        'ground_truth_by_class': defaultdict(int)
    }

    # Evaluate each image
    for image_name in tqdm(dataset.images):
        # Get ground truth
        gt_annotations = dataset.annotations[image_name]
        gt_boxes = gt_annotations.xyxy
        gt_classes = gt_annotations.class_id

        # Update ground truth class counts
        for cls in gt_classes:
            metrics['ground_truth_by_class'][cls] += 1

        # Get predictions
        image = dataset.images[image_name]
        results = model(image)[0]
        pred_boxes = results.boxes.xyxy.cpu().numpy()
        pred_classes = results.boxes.cls.cpu().numpy()
        pred_conf = results.boxes.conf.cpu().numpy()

        # Filter by confidence threshold
        mask = pred_conf >= conf_threshold
        pred_boxes = pred_boxes[mask]
        pred_classes = pred_classes[mask]
        pred_conf = pred_conf[mask]

        # Calculate IoU matrix
        if len(pred_boxes) > 0 and len(gt_boxes) > 0:
            iou_matrix = calculate_iou(pred_boxes, gt_boxes)

            # Match predictions to ground truth
            matched_indices = []
            for pred_idx, pred_class in enumerate(pred_classes):
                best_iou = 0
                best_gt_idx = -1

                for gt_idx, gt_class in enumerate(gt_classes):
                    if gt_idx not in matched_indices and pred_class == gt_class:
                        iou = iou_matrix[pred_idx, gt_idx]
                        if iou >= iou_threshold and iou > best_iou:
                            best_iou = iou
                            best_gt_idx = gt_idx

                if best_gt_idx >= 0:
                    matched_indices.append(best_gt_idx)
                    metrics['true_positives'] += 1
                    metrics['confidences'].append(pred_conf[pred_idx])
                    metrics['predictions_by_class'][pred_class].append(True)
                    metrics['prediction_confidences_by_class'][pred_class].append(pred_conf[pred_idx])
                else:
                    metrics['false_positives'] += 1
                    metrics['predictions_by_class'][pred_class].append(False)
                    metrics['prediction_confidences_by_class'][pred_class].append(pred_conf[pred_idx])

            metrics['false_negatives'] += len(gt_boxes) - len(matched_indices)
        else:
            metrics['false_positives'] += len(pred_boxes)
            metrics['false_negatives'] += len(gt_boxes)

    # Calculate overall metrics
    total_predictions = metrics['true_positives'] + metrics['false_positives']
    total_ground_truth = metrics['true_positives'] + metrics['false_negatives']

    precision = metrics['true_positives'] / total_predictions if total_predictions > 0 else 0
    recall = metrics['true_positives'] / total_ground_truth if total_ground_truth > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Calculate per-class metrics
    class_metrics = {}
    for class_id in metrics['ground_truth_by_class'].keys():
        predictions = metrics['predictions_by_class'][class_id]
        ground_truth = metrics['ground_truth_by_class'][class_id]
        prediction_confidences = np.array(metrics['prediction_confidences_by_class'][class_id])
        prediction_labels = np.array(predictions).astype(int)

        if len(predictions) > 0:
            class_tp = sum(predictions)
            class_fp = len(predictions) - class_tp
            class_fn = ground_truth - class_tp

            class_precision = class_tp / (class_tp + class_fp) if (class_tp + class_fp) > 0 else 0
            class_recall = class_tp / (class_tp + class_fn) if (class_tp + class_fn) > 0 else 0
            class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0

            # Calculate AP50 for the class
            class_ap50 = calculate_ap50(prediction_confidences, prediction_labels, ground_truth)

            class_metrics[class_id] = {
                'precision': class_precision,
                'recall': class_recall,
                'f1': class_f1,
                'ap50': class_ap50,
                'support': ground_truth
            }

    # Visualize results
    plt.figure(figsize=(20, 5))

    # Plot 1: Confidence Distribution
    plt.subplot(141)
    if metrics['confidences']:
        sns.histplot(metrics['confidences'], bins=20)
        plt.title('Confidence Distribution\nfor True Positives')
        plt.xlabel('Confidence Score')
        plt.ylabel('Count')

    # Plot 2: Overall Metrics
    plt.subplot(142)
    overall_metrics = [precision, recall, f1]
    plt.bar(['Precision', 'Recall', 'F1-Score'], overall_metrics)
    plt.title('Overall Model Performance')
    plt.ylim(0, 1)

    # Plot 3: Per-class F1 Scores
    plt.subplot(143)
    class_f1_scores = [metrics['f1'] for metrics in class_metrics.values()]
    plt.bar(list(class_metrics.keys()), class_f1_scores)
    plt.title('F1-Score by Class')
    plt.xlabel('Class ID')
    plt.ylabel('F1-Score')
    plt.ylim(0, 1)

    # Plot 4: Per-class AP50
    plt.subplot(144)
    class_ap50_scores = [metrics['ap50'] for metrics in class_metrics.values()]
    plt.bar(list(class_metrics.keys()), class_ap50_scores)
    plt.title('AP50 by Class')
    plt.xlabel('Class ID')
    plt.ylabel('AP50')
    plt.ylim(0, 1)

    plt.tight_layout()
    plt.show()

    # Print detailed metrics
    print("\nOverall Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"True Positives: {metrics['true_positives']}")
    print(f"False Positives: {metrics['false_positives']}")
    print(f"False Negatives: {metrics['false_negatives']}")

    print("\nPer-class Metrics:")
    for class_id, metrics in class_metrics.items():
        print(f"\nClass {class_id}:")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-Score: {metrics['f1']:.4f}")
        print(f"AP50: {metrics['ap50']:.4f}")
        print(f"Support: {metrics['support']}")

    return metrics

# Example usage
HOME = "/content"  # Adjust this to your environment
model_path = "/content/YOLO200.pt"

# Create dataset
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="/content/datasets/Drone3-18/test/images",
    annotations_directory_path="/content/datasets/Drone3-18/test/labels",
    data_yaml_path="/content/datasets/Drone3-18/data.yaml"
)

# Evaluate model
metrics = evaluate_yolo_model(model_path, dataset)

## Testing YOLO model on real images

In [ ]:
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="/content/datasets/DroneReal-2-4/test/images",
    annotations_directory_path="/content/datasets/DroneReal-2-4/test/labels",
    data_yaml_path="/content/datasets/DroneReal-2-4/data.yaml"
)

# Evaluate model
metrics = evaluate_yolo_model(model_path, dataset)